In [1]:
import dlib
import dlib
import cv2
from imutils import face_utils
import argparse
from utils import *
import os
import glob
import torch

In [71]:
WALK = os.walk('/media/data1/mhkim/FAKEVV_hasam/train/FRAMES/real_A_fake_MergedC_XRAY/FAKE_2TH/')

In [5]:
WALK = os.walk('/media/data1/mhkim/FAKEVV_hasam/test/FRAMES/real_A_fake_others/REAL/')

In [6]:
list_path_fake = []
for (a,b,c) in WALK:
    for _c in c:
        if '.jpg' in _c:
            list_path_fake.append(os.path.join(a,_c))
list_path_fake.sort()

In [7]:
len(list_path_fake)

2418

In [14]:
def parse_args():
    desc = "Data preprocessing for Deep3DRecon."
    parser = argparse.ArgumentParser(description=desc)

    parser.add_argument('--img_path', type=str, default='./input', help='original images folder')
    parser.add_argument('--save_path', type=str, default='./processed_data', help='custom path to save proccessed images and labels')


    return parser.parse_args()

def midpoint(p1, p2):
    coords = (p1[0] + p2[0]) / 2, (p1[1] + p2[1]) / 2
    return [int(x) for x in coords]

def preprocessing(list_path_fake, find_main_point = False):
    dict_landmark = {}
    img_list = list_path_fake

    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

    for file in img_list:
        img = cv2.imread(file) #img must be cropped into face
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 0)

        if len(rects) > 0:
            for rect in rects:
                if find_main_point:
                    x = rect.left()
                    y = rect.top()
                    w = rect.right()
                    h = rect.bottom()
                    img = img[y:h, x:w]

                shape = predictor(img, rect)
                shape_np = face_utils.shape_to_np(shape).tolist()
                if find_main_point:
                    left_eye = midpoint(shape_np[36], shape_np[39])
                    right_eye = midpoint(shape_np[42], shape_np[45])
                    features = [left_eye, right_eye, shape_np[33], shape_np[48], shape_np[54]]
                    dict_landmark[file] = features
                else:
                    dict_landmark[file] = shape_np
    return dict_landmark

dict_land = {}
dict_land = preprocessing(list_path_fake)

In [9]:
len(dict_land)

2383

# DICT -> JSON 저장하기

In [10]:
import json
with open('real_landmark_test.json','w', encoding='utf-8') as make_file:
    json.dump(dict_land,make_file,indent='\t')

In [13]:
import json
with open('real_landmark_test.json', 'r') as f:
    landmarks_record =  json.load(f)
len(landmarks_record)

2383

In [108]:
temp = []
temp.append(landmarks_record[list(landmarks_record)[0]])


In [104]:
for i in landmarks_record:
    print(i)
    break

/media/data1/mhkim/FAKEVV_hasam/train/FRAMES/real_A_fake_MergedC/FAKE/FAKE_FACESWAP_C/00001_id00634_z5t9oJizDQg_faceswap_343_00005.jpg


In [102]:
np.array(temp[0])

array([[ 45,  55],
       [ 45,  71],
       [ 45,  87],
       [ 46, 103],
       [ 50, 119],
       [ 58, 133],
       [ 71, 145],
       [ 87, 154],
       [104, 159],
       [122, 159],
       [139, 152],
       [152, 141],
       [162, 128],
       [167, 114],
       [171,  99],
       [175,  84],
       [176,  68],
       [ 65,  40],
       [ 75,  35],
       [ 87,  34],
       [ 99,  38],
       [109,  44],
       [127,  47],
       [138,  43],
       [149,  43],
       [160,  45],
       [168,  52],
       [117,  57],
       [116,  67],
       [115,  77],
       [115,  88],
       [103,  95],
       [107,  97],
       [112, 100],
       [118,  99],
       [123,  98],
       [ 77,  54],
       [ 85,  51],
       [ 93,  52],
       [100,  59],
       [ 91,  59],
       [ 83,  58],
       [132,  63],
       [140,  58],
       [148,  59],
       [155,  64],
       [148,  66],
       [140,  65],
       [ 82, 112],
       [ 93, 109],
       [104, 109],
       [111, 111],
       [118,

In [55]:
list_temp = []
for a in zip(landmarks_record):
    list_temp.append(a[0])
    

In [58]:
list_temp2 = []
for a in zip(landmarks_record):
    list_temp2.append(a[0])
    

In [18]:
origin = list(landmarks_record)

In [59]:
temp = ['ab','abc']
'abc' in temp

True

In [7]:
import glob
xray = glob.glob('/media/data1/mhkim/FAKEVV_hasam/train/FRAMES/real_A_fake_MergedC_XRAY/FAKE_2TH/*.jpg')

In [36]:
for i in range(len(xray)):
    xray[i] = xray[i].split('/')[-1]

In [41]:
for i in range(len(origin)):
    splited = origin[i].split('/')
    origin[i] = splited[-2] +'_'+ splited[-1]

In [33]:
origin[1].split('/')[-1]

'00001_id00634_z5t9oJizDQg_faceswap_343_00010.jpg'

In [26]:
import numpy as np
np.array(xray)[:,1].split('/')[-1]

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [43]:
set_origin = set(origin)
set_xray = set(xray)
print(len(set_origin), len(set_xray))

62579 60084


In [44]:
set_diff = set.difference(set_origin, set_xray)

In [47]:
diff = list(set_diff)

In [56]:
diff[0][:diff[0].index('C_')+1]

'FAKE_W2L_C'

In [52]:
diff[0][diff[0].index('C_')+2:]

'00305_id00431_DyRvPN6rEfU_wavtolip_1424_00075.jpg'

In [48]:
diff.split()

['FAKE_W2L_C_00305_id00431_DyRvPN6rEfU_wavtolip_1424_00075.jpg',
 'FAKE_W2L_C_00288_id01035_6EWG3Ywu3jg_wavtolip_834_00090.jpg',
 'FAKE_W2L_C_00420_id00262_3p7sRWAAh1Q_wavtolip_2881_00080.jpg',
 'FAKE_W2L_C_00417_id06254_8VSfI86Fua4_wavtolip_3715_00040.jpg',
 'FAKE_W2L_C_00322_id00358_vsVwGKy79kM_wavtolip_2672_00050.jpg',
 'FAKE_W2L_C_00322_id00535_BhvTIJ-YG5s_wavtolip_2673_00015.jpg',
 'FAKE_W2L_C_00305_id03696_Kz45DNXKLXI_wavtolip_1429_00015.jpg',
 'FAKE_W2L_C_00340_id00368_KohcGDsg29U_wavtolip_2583_00035.jpg',
 'FAKE_W2L_C_00466_id04774_k3yOcTA95dg_wavtolip_1775_00085.jpg',
 'FAKE_W2L_C_00288_id01239_gJvquKo7kbA_wavtolip_838_00095.jpg',
 'FAKE_W2L_C_00340_id00192_By3wxX0dyDw_wavtolip_2581_00090.jpg',
 'FAKE_W2L_C_00470_id01052_Kf4nlIEHfaU_wavtolip_2428_00185.jpg',
 'FAKE_W2L_C_00322_id00709_Rh23ELMGsIQ_wavtolip_2674_00030.jpg',
 'FAKE_W2L_C_00412_id08313_t6WZ9KCaXBM_wavtolip_3449_00045.jpg',
 'FAKE_W2L_C_00466_id06467_bN8jVda0Byw_wavtolip_1778_00025.jpg',
 'FAKE_W2L_C_00317_id07165_

In [45]:
len(set_diff)

2495

In [18]:
temp = list(landmarks_record.keys())

In [19]:
temp[0].split('/')[-1]

'00001_id00634_z5t9oJizDQg_faceswap_343_00005.jpg'

In [47]:
img_list = (glob.glob('/media/data1/mhkim/FAKEVV_hasam/train/FRAMES/real_A_fake_MergedC/REAL/REAL_A/*')[:2])
img_list
# img_list = sorted(glob.glob(image_path + '/' + '*.png'))

['/media/data1/mhkim/FAKEVV_hasam/train/FRAMES/real_A_fake_MergedC/REAL/REAL_A/00044_27_00060.jpg',
 '/media/data1/mhkim/FAKEVV_hasam/train/FRAMES/real_A_fake_MergedC/REAL/REAL_A/00010_15_00239.jpg']

In [49]:
import glob
temp_list = glob.glob('/media/data1/mhkim/FAKEVV_hasam/train/FRAMES/real_A_fake_MergedC/REAL/REAL_A/*')

In [29]:
aa = [1,2,3]

In [31]:
np.array(aa).astype(np.float32)

array([1., 2., 3.], dtype=float32)

In [56]:
cnt = 0
for (a,b,c) in os.walk('/media/data1/mhkim/FAKEVV_hasam/train/FRAMES/real_A_fake_MergedC/REAL/REAL_A/'):
    for _b in b:
#         print(os.path.join(a,_b))
        cnt += len(os.listdir(os.path.join(a,_b)))


In [57]:
cnt

63678

In [60]:
import glob
temp_list = glob.glob('/media/data1/mhkim/FAKEVV_hasam/train/FRAMES/real_A_fake_MergedC/REAL/REAL_A/*.jpg')

In [65]:
temp_list[0].split('/')[-1][:-4]

'00044_27_00060'

In [61]:
len(temp_list)

63678